# データ処理方法 -pandas-
こちらはpandasのコードを書く

## ライブラリのインポート

In [1]:
import numpy as np
import pandas as pd

In [2]:
print(pd.__version__)
print(np.__version__)

2.2.2
2.0.1


## データの読み込み
今回は3種類のデータを用意
* データ1
    * 100万行4列
* データ2
    * 50万行4列
    * データ1と同じ列
* 重みデータ
    * データ1とデータ2に存在する2つの列("group1", "group2")の各値に対する重み値を保存しているデータ


In [3]:
%%timeit
data1 = pd.read_csv("./sample_data/data1.csv")

875 ms ± 113 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
data1 = pd.read_csv("./sample_data/data1.csv")
data2 = pd.read_csv("./sample_data/data2.csv")
weight_data = pd.read_excel("./sample_data/weight_data.xlsx")

## データの確認
* サイズの確認
    * `shape`
* データの表示
    * `head()`
    * `tail()`
    * `sample()`
* 要約量の表示
    * `describe()`

In [5]:
data1.shape

(1000000, 4)

In [6]:
data1.head(n=10)

,id,group1,group2,value
0,1,C,X,0.875341
1,2,C,Z,-1.274481
2,3,A,Z,0.299180
3,4,C,Z,-1.326054
4,5,B,Z,0.166297
5,6,A,Z,-0.407868
6,7,C,Y,-1.155394
7,8,A,Y,0.717269
8,9,B,Z,-0.385354
9,10,B,X,-1.148564


In [7]:
data2.sample(n=5)

,id,group1,group2,value
25693,1025694,A,Y,-1.048057
202791,1202792,B,Z,-0.498988
234507,1234508,B,Y,1.426105
34083,1034084,B,Y,0.045332
306588,1306589,B,Z,0.143367


In [8]:
weight_data.tail(n=9)

,group1,group2,weight
0,A,X,5
1,A,Y,6
2,A,Z,1
3,B,X,4
4,B,Y,9
5,B,Z,8
6,C,X,5
7,C,Y,4
8,C,Z,4


In [9]:
data1.describe()

,id,value
count,1000000.000000,1000000.000000
mean,500000.500000,0.000969
std,288675.278932,1.001348
min,1.000000,-4.729759
25%,250000.750000,-0.675544
50%,500000.500000,0.000248
75%,750000.250000,0.675407
max,1000000.000000,4.715991


## データ操作
* 列の操作
* 行の操作
* グループ化

### 列の操作

In [10]:
# 列の抽出
# 正規表現で列を抽出
data1.filter(regex="^gro.*$", axis=1)
data1.loc[:, data1.columns.str.contains("^gro.*$")]

,group1,group2
0,C,X
1,C,Z
2,A,Z
3,C,Z
4,B,Z
...,...,...
999995,A,X
999996,B,Y
999997,A,Y
999998,A,Z


In [11]:
# ある列以外を抽出する
data1.loc[:, ~data1.columns.str.contains("^gro.*$")]

,id,value
0,1,0.875341
1,2,-1.274481
2,3,0.299180
3,4,-1.326054
4,5,0.166297
...,...,...
999995,999996,0.503502
999996,999997,-1.031320
999997,999998,-0.157509
999998,999999,0.811663


In [12]:
data1.loc[:, "group1"].value_counts()

group1
C    323287
B    322250
A    322207
Name: count, dtype: int64

### 行の操作

In [13]:
# 行の抽出
# クエリの中で扱える列名はバッククォートで囲むと，列名に空白等があっても問題ない
data1.query("group1 == 'A'")
# データフレーム名を何度も指定する必要あり
data1.loc[data1["group1"] == "A"]

,id,group1,group2,value
2,3,A,Z,0.299180
5,6,A,Z,-0.407868
7,8,A,Y,0.717269
12,13,A,Z,-0.299052
13,14,A,Z,0.623657
...,...,...,...,...
999982,999983,A,X,-0.613297
999984,999985,A,Z,0.457641
999995,999996,A,X,0.503502
999997,999998,A,Y,-0.157509


In [14]:
# 複数条件
foo = "X"
data1.query("group1 == 'A' and group2 == @foo")

,id,group1,group2,value
32,33,A,X,-0.350490
38,39,A,X,2.419110
46,47,A,X,0.155541
47,48,A,X,0.567384
60,61,A,X,-0.182846
...,...,...,...,...
999953,999954,A,X,2.055218
999954,999955,A,X,0.465499
999967,999968,A,X,-0.480700
999982,999983,A,X,-0.613297


In [15]:
# リスト条件を使う
data1.query("group1 in ['A', 'C']")

,id,group1,group2,value
0,1,C,X,0.875341
1,2,C,Z,-1.274481
2,3,A,Z,0.299180
3,4,C,Z,-1.326054
5,6,A,Z,-0.407868
...,...,...,...,...
999993,999994,C,NaN,-0.180725
999994,999995,C,X,1.733561
999995,999996,A,X,0.503502
999997,999998,A,Y,-0.157509


In [16]:
# 欠損値を表示する
data1[data1[["group1", "group2"]].isnull().all(axis=1)]

,id,group1,group2,value
284,285,NaN,NaN,-0.619982
816,817,NaN,NaN,-1.530416
1206,1207,NaN,NaN,-0.837263
3682,3683,NaN,NaN,-0.010933
5072,5073,NaN,NaN,0.218643
...,...,...,...,...
996681,996682,NaN,NaN,-0.574811
997954,997955,NaN,NaN,-0.833321
998193,998194,NaN,NaN,-0.306645
998585,998586,NaN,NaN,1.103169


In [17]:
# デフォルトでは，欠損値は除外される
data1.groupby("group1", dropna=False).mean("value")

,id,value
group1,,
A,500322.403219,-0.000072
B,500194.133216,0.001216
C,499566.821901,0.002538
NaN,499197.074808,-0.006814


## 前処理
* 重複の削除
* データフレームの結合
* 欠損値処理
* 新しい列の作成
* 縦長データ <-> 横長データの変換

### 重複の削除
* 重複の削除
    * `drop_duplicate()`
    * 特定の列のみの重複を削除したい場合，`subset`引数を

In [18]:
df = pd.DataFrame({"A": ["X", "X", "Y"], "B": [1, 1, 1]})
df.drop_duplicates()

,A,B
0,X,1
2,Y,1


### データフレームの結合
* 縦に結合する
    * `pd.concat()`
* 横に結合する
    * `pd.merge()`

In [19]:
# 縦に結合する
data12 = pd.concat([data1, data2])
data12.shape

(1500000, 4)

In [20]:
data12.head()

,id,group1,group2,value
0,1,C,X,0.875341
1,2,C,Z,-1.274481
2,3,A,Z,0.299180
3,4,C,Z,-1.326054
4,5,B,Z,0.166297


In [21]:
# weight_dataを結合するために，データを確認する
weight_data.head(n=10)

,group1,group2,weight
0,A,X,5
1,A,Y,6
2,A,Z,1
3,B,X,4
4,B,Y,9
5,B,Z,8
6,C,X,5
7,C,Y,4
8,C,Z,4


In [22]:
# 横に結合する
# キーはgroup1列，group2列同士で行う
all_data = pd.merge(left=data12, 
                    right=weight_data,
                    left_on=["group1", "group2"],
                    right_on=["group1", "group2"],
                    how="left")

In [23]:
all_data.shape

(1500000, 5)

In [24]:
all_data.sample(n=5)

,id,group1,group2,value,weight
635785,635786,B,X,2.152816,4.0
1308794,1308795,A,Z,-0.916148,1.0
725663,725664,A,Z,-0.100977,1.0
224602,224603,A,Y,0.001222,6.0
1378801,1378802,C,X,-1.437159,5.0


In [25]:
# 欠損値を持つ行を表示．
# 結合方法を左結合にしたため，左で欠損を持つ行(厳密には，右側のデータにない結合キー)も残っている
all_data[all_data["group1"].isnull()]

,id,group1,group2,value,weight
17,18,NaN,Y,0.405518,NaN
19,20,NaN,Y,0.105258,NaN
66,67,NaN,Z,0.588424,NaN
67,68,NaN,Y,1.016913,NaN
77,78,NaN,Y,-0.061921,NaN
...,...,...,...,...,...
999825,999826,NaN,Z,0.200896,NaN
999931,999932,NaN,Y,-0.607453,NaN
999951,999952,NaN,Y,-1.488471,NaN
999966,999967,NaN,Z,-0.100375,NaN


### 欠損値処理
* 欠損値の除去
    * `dropna`
* 欠損値の補完
    *     

In [26]:
sample_data = pd.DataFrame(
    {
        "a": [None, None, None, 3],
        "b": [1, 2, None, 1],
        "c": [1, None, None, 1],
    }
)
sample_data

,a,b,c
0,NaN,1.0,1.0
1,NaN,2.0,NaN
2,NaN,NaN,NaN
3,3.0,1.0,1.0


In [27]:
# b列が欠損である行を削除する
sample_data.dropna(subset="b")

,a,b,c
0,NaN,1.0,1.0
1,NaN,2.0,NaN
3,3.0,1.0,1.0


In [28]:
# 1列でも欠損値を持つ行を削除する
sample_data.dropna()

,a,b,c
3,3.0,1.0,1.0


In [29]:
# すべての列の値が欠損である行を削除する
sample_data.dropna(how="all")

,a,b,c
0,NaN,1.0,1.0
1,NaN,2.0,NaN
3,3.0,1.0,1.0


In [30]:
# 全ての行が欠損である列を削除する
sample_data2 = pd.DataFrame(
    {
        "a": [None, None, None],
        "b": [1, 2, None],
        "c": [1, None, None],
    }
)
print(sample_data2)
sample_data2.dropna(axis=1, how="all")

      a    b    c
0  None  1.0  1.0
1  None  2.0  NaN
2  None  NaN  NaN


,b,c
0,1.0,1.0
1,2.0,NaN
2,NaN,NaN


In [31]:
# 欠損値の補完
sample_data.fillna(-99)

,a,b,c
0,-99.0,1.0,1.0
1,-99.0,2.0,-99.0
2,-99.0,-99.0,-99.0
3,3.0,1.0,1.0


In [32]:
# 欠損値を各列の最大値で補完
sample_data.fillna(value=sample_data.max())

,a,b,c
0,3.0,1.0,1.0
1,3.0,2.0,1.0
2,3.0,2.0,1.0
3,3.0,1.0,1.0


In [33]:
# 今後の処理で欠損値があるとよくないので，最初のデータの欠損値を削除する
all_data = all_data.dropna()

### 新しい列の作成
* 直接代入する方法
* `assign()`メソッドを持ちる方法

In [34]:
data1["group1_2"] = data1["group1"] + "_" + data1["group2"]
data1.assign(value_x2 = data1["value"]*2,)
data1

,id,group1,group2,value,group1_2
0,1,C,X,0.875341,C_X
1,2,C,Z,-1.274481,C_Z
2,3,A,Z,0.299180,A_Z
3,4,C,Z,-1.326054,C_Z
4,5,B,Z,0.166297,B_Z
...,...,...,...,...,...
999995,999996,A,X,0.503502,A_X
999996,999997,B,Y,-1.031320,B_Y
999997,999998,A,Y,-0.157509,A_Y
999998,999999,A,Z,0.811663,A_Z


### 縦長データ <-> 横長データの変換
* 縦長データ -> 横長データ
    * `pivot()`
* 横長データ -> 縦長データ
    * `melt()`

In [35]:
sample_data = pd.DataFrame(
    {
        "id": [1, 1, 1, 2, 2, 2],
        "cat": ["A", "B", "C", "A", "B", "C"],
        "value1": [10, 12, 14, 16, 18, 20],
        "value2": [1, 2, 4, 6, 8, 10],
    }
)
sample_data

,id,cat,value1,value2
0,1,A,10,1
1,1,B,12,2
2,1,C,14,4
3,2,A,16,6
4,2,B,18,8
5,2,C,20,10


In [36]:
# ピボット
sample_data.pivot(columns=["cat"], index=["id"], values=["value1", "value2"])

value1         value2       
cat      A   B   C      A  B   C
id                              
1       10  12  14      1  2   4
2       16  18  20      6  8  10

In [37]:
# ピボット解除
sample_data.melt(id_vars=["id", "cat"], value_vars=["value1", "value2"])

,id,cat,variable,value
0,1,A,value1,10
1,1,B,value1,12
2,1,C,value1,14
3,2,A,value1,16
4,2,B,value1,18
5,2,C,value1,20
6,1,A,value2,1
7,1,B,value2,2
8,1,C,value2,4
9,2,A,value2,6
